<img align="left" src="https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/tapi-logo-small.png" />

This notebook free for educational reuse under [Creative Commons CC BY License](https://creativecommons.org/licenses/by/4.0/).

Created by [Xanda Schofield](https://www.cs.hmc.edu/~xanda) for the 2022 Text Analysis Pedagogy Institute, with support from the [National Endowment for the Humanities](https://neh.gov), [JSTOR Labs](https://labs.jstor.org/), and [University of Arizona Libraries](https://new.library.arizona.edu/).

For questions/comments/improvements, email xanda@cs.hmc.edu.<br />
____

# Text Data Curation 2

This is lesson 2 of 3 in the educational series on Text Data Curation. This notebook is intended to introduce the basics of treating text documents as data and how to store and filter those documents.

**Audience:** `Learners` / `Researchers`

**Use case:** [`How-To`](https://constellate.org/docs/documentation-categories#howtoproblemoriented) 

**Difficulty:** `Intermediate`
Assumes users are familiar with Python and have been programming for 6+ months. Code makes up a larger part of the notebook and basic concepts related to Python are not explained.

**Completion time:** `90 minutes`

**Knowledge Required:** 
* Python basics (variables, flow control, functions, lists, dictionaries)
* How Python libraries work (installation and imports)

**Knowledge Recommended:**
* Basic file operations (open, close, read, write)
* Basics of what regular expressions are (pattern recognizers for text)

**Learning Objectives:**
After this lesson, learners will be able to:
1. Recognize and remove unwanted text using regular expressions
2. Produce feature counts of words from text using a CountVectorizer
3. Identify bigrams with nltk
4. Normalize tokens in text using spaCy
___

# Required Python Libraries
* matplotlib
* nltk
* numpy
* scikit-learn
* spacy

## Install Required Libraries

In [ ]:
### Install Libraries ###

# Get spacy and the English web data language model
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install scikit-learn

# Get nltk set up
import nltk
nltk.download('punkt')

In [ ]:
### Import Libraries ###
# Built into Python
from collections import Counter
import re

# External imports
from matplotlib import pyplot as plt
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
from nltk.tokenize import RegexpTokenizer
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
import spacy

# Required Data

**Data Format:** 
* Python data archive (built into sklearn)

**Data Source:**
* [20 Newsgroups via Scikit-Learn](https://scikit-learn.org/stable/datasets/real_world.html#the-20-newsgroups-text-dataset)

**Data Quality/Bias:**
This dataset is originally made from retrieving 1000 messages each from 20 different Usenet newsgroups. Because of cross-posting between newsgroups and repetition, the original dataset is almost never used; this version of the dataset is trimmed to ~18,000 messages across the newsgroups. While the utilities for the dataset allow you to remove headers, quotes, and footers/signature lines, the dataset still contains a significant amount of noise, as well as some views and statements that some may find disturbing or offensive.


## Download Required Data

In [ ]:
### Grab just the space messages to start
newsgroup_data = fetch_20newsgroups(categories=['sci.space'])

# Introduction

In the previous lesson, we looked at the metadata of a Spanish poems "meta"-dataset and explored ways to store and explore that data, noticing things about encodings, author frequency, and duplication. In this section, we're going to look more at unstructured text directly and how we can pre-process that text. We'll use 20 Newsgroups, and we'll talk about checking for odd formatting and things to remove, counting tokens from text, and how we can modify the forms of those tokens when producing count features.

# Lesson

A lot of popular text analysis represent text using a bag-of-words model: they provide counts of words or phrases (which we'll call *lexical features*) as a way to quantitatively distinguish what shows up rarely, often, or together with other things. To make these processes work, we want to be able to generate counts of lexical features that both differentiate documents and that show up often enough to find consistency.

## Our dataset: 20 Newsgroups

To help explore this, we'll use 20 Newsgroups. This dataset dates back to the 1990s, where it became popular in the machine learning community as a classification dataset. The original premise was to collect 20,000 messages, 1,000 from each of twenty different newsgroups on Usenet, and to train machine learning models to predict which of the newsgroups a post was from. (If you're not familiar with Usenet, think of it as the 80's version of Reddit: newsgroups are devoted to particular topics, and messages were posted to everyone subscribed to that topic.)

As it turns out, some messages were cross-posted on multiple threads. With those removed, the dataset most people use has 18,828 posts. The newsgroups include topics ranging from different religions and atheism to motorcycles, science, sports, and gun rights. To keep things light, and since we're not really interested in actually classifying text today, I'll just pull the messages from `sci.space`.

To make our lives easier to start, I'm going to make a small helper function to allow me to pull a message by its position among the 593 messages we will have to look at and print a reasonable sample of it. (They're pretty long.) By default, this will print the first 1000 characters of the newsgroup data I have pulled, but I've added some keyword arguments so I can do some different things with it later.

In [ ]:
def data_by_idx(idx, full=False, n_chars=1000, data=newsgroup_data.data):
    if full or len(data[idx]) <= n_chars:
        return data[idx]
    else:
        return data[idx][:n_chars] + ' ... [ {} more characters ]'.format(len(data[idx]) - n_chars)

In [ ]:
# print the first 10 documents
for i in range(10):
    print(data_by_idx(i))

Now we get our first look at these documents. And they are a mess! One thing we notice at the top of these is that they have headers (`From:`, `Subject:`, etc.) which aren't really the bodies of these messages, but are included by default. Some of this could be useful to us - they actually include the total lengths of the bodies of the messages, which could be useful for grabbing text in some settings, but it's not great. We also notice there are quotes (`In article...`), email signatures, and other fun things.

These are the sorts of things that show up in documents that started out digitally. Usually, the text is pretty clean (that is, while people may make typos, we're getting an accurate version of what people wrote), but information about how a post was made, formatting details, or other bits of customization will often be in the bodies of this text. Now, in certain circumstances, that may be exactly what you're interested in - but for most text analysis, we probably want to filter the text down to the specific pieces written directly by the author for that post instead.

Let's start trying to just remove the quotes. We can see quotes seem to start with `>` or `|>` in these lines, so we could elect to just delete all of the lines that start with that. I have two different ways we can do that below. The first uses the Python `startswith` string method (which isn't super flexible but straightforward for detecting exact beginnings of strings). The second one uses a regular expression to identify lines with one of these two beginnings and `sub`stitute those lines with an empty line.

In [ ]:
def remove_quotes_a(doc):
    # delete lines that start with > or |>
    lines = doc.split('\n')
    lines_to_keep = [l for l in lines if not (l.startswith('>') or l.startswith('|>'))]
    return '\n'.join(lines_to_keep)


def remove_quotes_b(doc):
    # replace a sequence of lines that starts with > or |> with one new line
    # I use pythex.org to troubleshoot these for Python
    return re.sub(r'\n(\|?>+[^\n]*\n)+', '\n', doc)

In [ ]:
for i in range(10):
    print(remove_quotes_a(data_by_idx(i, full=True)))

**Exercise:** Look at the two functions above. What are the pros and cons of the two methods above? How would you adjust them to handle lines that start with one or more spaces before the quote symbol `>`?

Now, there are fairly sophisticated regular expressions and other manual steps people have done to remove a lot more than quotes. (Headers are moderately straightforward; email signatures are tougher.) Lucky for us, `scikit-learn` has this effort built in, so we're going to use that instead of starting from scratch.

Note: I'm going to make a new variable for this data. If this was a big dataset, this might put too much data into the memory of this program at once - but since these are smaller, it's a good idea for me to name these separately so I can remember which code I wrote for which version of this data. If I write something later that assumes the data is clean, I want to be able to read through the notebook and see that. Let's see how this looks now.

In [ ]:
# Using a new name to avoid conflicts
clean_newsgroup_data = fetch_20newsgroups(categories=['sci.space'], remove=('headers', 'footers', 'quotes'))
for i in range(10):
    print('--- MESSAGE', i, '---')
    print(data_by_idx(i, data=clean_newsgroup_data.data))
    print()

We're now missing some context of this text, so if we wanted to read and interpret these messages, this might be an issue. However, for a program, this is probably the content we want left: just the new authored natural language text of each message (mostly). But that's just for the ten documents we checked.

## Checking for outliers

Usually, when I get to a point where sample documents look reasonable, I'll do something small and automatic to check for outliers. For example, I might want to verify there aren't any ridiculously long messages that have gotten left in! So let's plot out the counts of characters and words in each of these documents!

Of course, I haven't defined what a word is yet, in part because it's an ambiguous term in this setting. When we want to count lexical features in text, we usually are interested in splitting text into working units containing a word, phrase, punctuation mark, or another kind of countable occurrence that be distinguished on its own, the atoms of our larger model of language. We call these elements *tokens*, and the process of splitting *tokenization*. We often talk about counting words, but when we use a word counting tool to make sure we're under a limit for a submitted text, we're counting the number of word *tokens*.

Tokens can share a *type*. For instance, let's look at how many times I said "we" in the last paragraph. Here are the possible candidates:
* When *we* want...
* ...in text, *we* usually...
* *We* call these...
* *We* often talk...
* ...when *we* use a...
* ...text, *we*'re counting...

We could count this as six instances of the word type *we*. Or, we could choose to distinguish every surface form of the word as a type: *We*, *we*, and *we're* could be three separate forms tokens could have that we treat differently. Quite often, we'll choose to *case fold*, or treat lowercase and uppercase words in English as the same type. More often than not, that's implemented by just lower-casing all of our text. Depending on our system for tokenizing, we may also treat apostrophes as separating words, or not, or something more complex. (The Punkt tokenizers would tokenize "didn't" as `["did", "n't"]`.)

Right now, what we're really looking for is a really basic check for whether this document looks like plausible text. Plausible text in English has space-separated words that are an average of 4-5 letters long. So we're just going to split text everywhere there's a whitespace character (spaces, newline characters) and use those as "words" for now, even though that's not really what we probably want to consider as our set of features later. Because we're relying on features of English, we should note that we'd need other techniques for other languages - the definition of a long document in unspaced Chinese is not going to benefit from splitting text on spaces and will probably be shorter in number of characters.

In [ ]:
plt.clf()  # clear previous plots

# Let's look at some histograms and see if there are any concerning outliers
char_counts = np.array([len(d) for d in clean_newsgroup_data.data])
word_counts = np.array([len(d.split()) for d in clean_newsgroup_data.data])
# can we keep track of how many lines are in each document as well?

# make a figure 3x as wide as it is tall
# this will allow us 3 subplots in one plot!
plt.figure(figsize=(12, 4))

# plot 1: character counts
plt.subplot(1, 3, 1) # this says this is the first of the plots in 1 row and 3 columns
plt.hist(x=char_counts, bins=50, color='red', label='Characters')
plt.xlabel("Character count")

# plot 2: word counts
plt.subplot(1, 3, 2) 
plt.hist(x=word_counts, bins=50, color='green', label='Words')
plt.xlabel("Word count")

# how would we make the third plot a blue plot of the number of lines in each?

plt.show()  # render the plot

**Exercise**: Right now there are two plots - but it might also be good to check if every document has a reasonable number of lines. How could we add a third plot (in blue, if you like) to do that?

Okay, there's nothing ridiculous, but we do see a sort of expected pattern of lots of shorter documents, and a few that are quite long. Just to make sure there's nothing weird going on in the larger documents, let's go ahead and print documents with more than 20,000 characters:

In [ ]:
long_documents = [doc for doc in clean_newsgroup_data.data if len(doc) > 20000]

# looking at an example
print("Number of long documents:", len(long_documents))
print()
print(data_by_idx(0, data=long_documents, full=True))

**Exercise** Check out a few documents in this list. Anything we notice about them that's unusual? Don't keep reading until you do this!

It looks like these documents include long, multi-chapter reports. The nice thing is these appear to be legitimate text (often, outliers I run into in digital corpora are signs that something got garbled). Unfortunately, they are quite long. In some tasks, it might make sense to split the text into smaller "pseudo-documents" that will be treated as separate by the algorithm. Sometimes, this will be done just by splitting the text in terms of number of lines or words (e.g. "break every 100 words"). This has a risk of splitting text in weird places, though, so I often will do one of two things depending on what the text can support:

1. If there are clear, nicely-formatted paragraphs with multiple lines in between, I'll use newline characters to split the text and then put one or more paragraphs in each pseudo-document so they're similar lengths. (Paragraphs have thematic unity that can be useful to preserve for text analysis - see this great Stanford Literary Lab pamphlet for some examples.) Since I use a lot of topic modeling in my work, and those tend to work well on documents that are a few hundred words long, one thing I might do is connect sequences of paragraphs that together have roughly at least 200 words, still using our heuristic of "stuff split by spaces". (We could improve on this using the tokenizers we'll talk about later, but it likely won't make a big difference on this document.)

In [ ]:
def pseudodocs_by_line_split(doc, min_words=200):
    paragraphs = doc.split('\n\n') 
    
    # Loop through the paragraphs to add them to the current doc
    pseudodocs = []
    current_pdoc = ''
    for par in paragraphs:
        current_pdoc += '\n\n' + par
        # if our pseudodoc is long enough, add it and reset
        if len(current_pdoc.split()) > min_words:
            pseudodocs.append(current_pdoc)
            current_pdoc = ''

    # important: we may have text left over at the end...if so, we'll
    # add it to the previous document
    pseudodocs[-1] += '\n\n' + current_pdoc
    return pseudodocs

In [ ]:
pdocs_1 = pseudodocs_by_line_split(long_documents[0])
for pdoc in pdocs_1:
    print(pdoc)
    print('\n~~~\n')

As you can see, this didn't work so well, in part because there are spaces on some of the empty lines, in part because there are indents in between some paragraphs. We could write a regular expression to handle that, or take another approach:

2. If there aren't good paragraph boundaries, I'll use something that detects sentences to make similar but not uniform length text passages. An example of how I might do that using a sentence tokenizer to make segments of at least 200 words is here:

In [ ]:
from nltk.tokenize import sent_tokenize

def pseudodocs_by_sentence(doc, min_words=200):
    sentences = sent_tokenize(doc)
    
    # Loop through the sentences to add them to the current doc
    pseudodocs = []
    current_pdoc = ''
    for sent in sentences:
        current_pdoc += ' ' + sent
        # if our pseudodoc is long enough, add it and reset
        if len(current_pdoc.split()) > min_words:
            pseudodocs.append(current_pdoc)
            current_pdoc = ''

    # important: we may have text left over at the end...if so, we'll
    # add it to the previous document
    pseudodocs[-1] += '\n\n' + current_pdoc
    return pseudodocs

In [ ]:
pdocs_2 = pseudodocs_by_sentence(long_documents[0])
for pdoc in pdocs_2:
    print(pdoc)
    print('\n~~~\n')

This looks a little more uniform in length. Unfortunately, we've got less thematic unity for our pieces of text, as some bridge multiple sections. We also lost our original line breaks doing this. However, if we anticipate that these documents are not representative of the whole corpus, spending that much time writing specific code to process those few documents is probably not a good use of time and will likely have limited effect on the final model.

You'll also notice there are some things that aren't quite cleaned up. For example, there are some extra headings that didn't get cleaned out in the second document. Let's go ahead and write some extra custom cleaning - since the formatting is quite specific, we should be able to remove these without too much inspection of documents to make sure we're not deleting something that isn't a header.

In [ ]:
def clean_text(text):
    text = re.sub(r'Archive-name: [^\n]*\n', '', text)  # remove header lines
    text = re.sub(r'Last-modified: [^\n]*\n', '', text) # remove header lines
    # how would you replace digits with a '#' symbol?
    return text

In [ ]:
print(clean_text(data_by_idx(1, full=True, data=long_documents)))

Okay, that seemed to help. Let's look at features.

## Generating tokens

A nice way to get a quick initial cut of some counts of different lexical features is using a `CountVectorizer`, a class from `scikit-learn` that takes a list of strings and (a) tokenizes them and (b) stores all of those counts in a matrix, or 2-D data structure of counts for each document and type. Rather than defining how it does this, let's start by taking a look by just feeding it the whole clean collection:

In [ ]:
cv = CountVectorizer(stop_words='english')
# can we adjust this to clean the data?
count_data = cv.fit_transform([clean_text(t) for t in clean_newsgroup_data.data])

print("Count matrix dimensions (documents x features):", count_data.shape)
print(count_data)

If you were expecting to see the 2D structure of a matrix (with 593 documents as rows and >13k word types as columns), this might be a bit confusing. When we count words in text, we usually expect most types in our vocabulary not to show up in most places, so if we made a full matrix of data, we would have a whole bunch of 0s and a much smaller proportion of 1s. To help cope with the storage inefficiency that would have, `scikit-learn` uses a *sparse* representation of the data: for each nonzero value in the matrix, it stores the row, column, and value. When we printed our matrix, we saw this format. 

Let's also look at the kinds of features we get out:

In [ ]:
features = cv.get_feature_names()
# note: soon to be deprecated, will be replaced with cv.get_feature_names_out
print(', '.join(features))

**Exercise** Skim the features. What sorts of things are treated as distinct types? Do you think this is using case folding? Is there anything you think should be excluded?

Right now, this system is using a regular expression to split text into alphanumeric sequences and punctuation sequences. We can see that it's including a lot of apparently random number sequences. These features aren't usually helpful for text analysis because they're too rare to be part of a meaningful pattern - they probably identify a specific author, thread, or message uniquely, but they don't tell us much else. Usually there are lots of these if we just count everything, so we may want to remove some of these unique elements.

The other type of feature we might want to remove are words that aren't distinguishing because they're too common. We usually refer to these as `stopwords`. Stopwords refer to words that are ignored by a text processing system because they're not relevant to the task at hand. There are canonical lists circulated for particular languages - for instance, `scikit-learn` has one built in for English:

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
print(ENGLISH_STOP_WORDS)

You might notice this list doesn't have any punctuation marks inside "can't" and "couldn't". Some lists don't have contractions at all, some have punctuation, and some are split by certain tokenizers into units like "can" and "n't". No stoplist is actually universal - they're adapted for choices of domain and tokenization scheme. If we wanted to remove these, we could use the argument `stop_words='english'` in the constructor of the `CountVectorizer`. But in general, I am skeptical about trusting these lists on their own, especially because they can remove interesting words (for instance, I think "everywhere" is a pretty interesting word!)

Of course, we can alter our tokenization scheme. While we can give an alternate regular expression directly to the `CountVectorizer`, I like using NLTK's tokenization support, as they have nice interfaces for tokenization. NLTK Tokenizer objects, like the `RegexpTokenizer` below, have a `tokenize` method that takes in text and returns a list of tokens. The `CountVectorizer` is happy to take a tokenizer function in as an argument, so you can use [something fancier](https://www.nltk.org/api/nltk.tokenize.html) than a regular expression from NLTK if you want!

In [ ]:
# this is a simpler tokenization option: one or more (+) letter characters (\w)
retok = RegexpTokenizer(r'[A-Za-z]+')

def tokenize_words(input_string):
    return retok.tokenize(input_string.lower())

One other cool thing our `CountVectorizer` supports is vocabulary curation. Rather than giving a stoplist, we can specify which word types we want to keep based on what proportion of documents they appear in. In this case, I'd like word types that show up in at least 5 documents but no more than 30% of all documents. I'll do this by setting `min_df=5` and `max_df=0.3`. Here, `df` is **document frequency**. If we give a positive integer for either of these, it'll use *absolute* document frequency, i.e. a count, as a threshold; if we give a float between 0 and 1, it will use that number as a proportion of the corpus, or *relative* document frequency threshold. Using this, we can prune our feature list dramatically, getting rid of both rare and common features.

In [ ]:
clean_cv = CountVectorizer(
    tokenizer=tokenize_words,    # use this function to split text
    min_df=5,                    # minimum # of documents to show up in
    max_df=0.3)                  # maximum proportion of documents to show up in
# can we adjust this to clean the data?
clean_count_data = clean_cv.fit_transform([t for t in clean_newsgroup_data.data])

# we should have many fewer features
print("New count matrix dimensions (documents x features):", clean_count_data.shape)

Let's see what features we now have:

In [ ]:
clean_features = clean_cv.get_feature_names()

print("Stoplist:", ', '.join(clean_cv.stop_words_))

for test_word in ['the', 'is', 'was', 'could', 'isn', 'msu', 'milky way']:
    print(test_word, "in features:", test_word in clean_features)

This looks like a lot more promising a list, as we have a lot of recognizable words!

People sometimes ask me exactly what numbers to use here. Like everything, it depends, but for English or other language where morphology isn't going to affect stopwords too much, I would set the max_df to something between .3 and .5, with the idea that a word that shows up in more than a third of the corpus is probably not very specific. The minimum is going to depend heavily on the number of documents in your corpus - for a multi-million document corpus, I would probably want something like 1000 or more, whereas for a few hundred documents, I would worry about going much above 5 or 10. Usually, you can visually inspect your token list and figure out if your terms are too specific or too vague. If you can't find a threshold that eliminates gibberish tokens, it's probably worth revisiting if you need to clean out formulaic portions of text, ideally by searching in your corpus for examples of the tokens whose types you can't get rid of.

**Exercise**: replace the tokenizer above with the function `nltk.tokenize.word_tokenize` (which you'll need to import). How does that change the behavior?

## Collocations: when one word won't do

A lot of domains have phrases whose meanings are distinct from their constituent words. For instance, while "climate" can refer to a community's social circumstance and "change" can refer to spare coins, "climate change" is probably not referring to either.

If we want to simply consider all possible two-token sequences, or even longer sequences (we call a sequence of length *n* an *ngram*), we can use our CountVectorizer directly to do so: `ngram_range=(1,3)`, for example, would allow us to specify we want to consider as features all sequences of one, two, or three tokens that appear in the corpus as possible features. The same frequency thresholds as before can be applied. Unfortunately, this tends to explode the size of our features in a way that usually includes way more ngrams we don't care about than ngrams we do. In some text processing circumstances, it makes sense to limit ourselves sequences of words for specific entities with a distinct meaning, like a noun phrase or figure of speech. We can automate that process using programs that count how often word pairs, or *bigrams*, appear compared to their single word, or *unigram* pieces. If they show up together way more than randomly, we might treat that as a collocation and a separate item in our lexicon.

Writing these finders from scratch is a pain. Lucky for us, NLTK actually has a BigramCollocationFinder built in, which allows us to specify a metric for finding collocations. I often use PMI, or [pointwise mutual information](https://en.wikipedia.org/wiki/Pointwise_mutual_information). Usually, it helps to couple this with something that discards particularly rare bigrams: for instance, if the only octopus mentioned in the corpus is blue, then we might find "blue octopus" marked as a high-PMI bigram even though it's rare.

**Exercise**: Look at the top 100 collocations found using PMI. What sorts of things do these bigrams capture? What happens if you change the frequency threshold from 5 to something else?

In [ ]:
tokenized_docs = [tokenize_words(clean_text(d)) for d in clean_newsgroup_data.data]

bigram_finder = BigramCollocationFinder.from_documents(tokenized_docs)
bigram_finder.apply_freq_filter(5)   # make sure all collocations have occured at least 5 times
collocations = bigram_finder.nbest(BigramAssocMeasures.pmi, 500) # find the top 500 bigrams by PMI

# print out each
for w1, w2 in collocations:
    print(w1, w2)

What do you do once you find these collocations? Well, you probably want to keep track of them and treat them as individual entities...but if you're using a downstream piece of code that will separate these bigrams into separate tokens again, that could be no good. Consequently, it's a surprisingly common process to take bigrams like these and replace them with a version connected with underscores, so those phrases can be treated as a single token going forward:

In [ ]:
colloc_docs = []

# iterate through docs making new versions combining collocations
for doc in tokenized_docs:
    new_doc = []
    i = 0
    while (i < len(doc) - 1):
        # if we find a collocation, add and advance by two words
        if tuple(doc[i:i+2]) in collocations:
            new_doc.append('_'.join(doc[i:i+2]))
            i += 2
        # otherwise, advance by one word
        else:
            new_doc.append(doc[i])
            i += 1

    # add the new doc
    colloc_docs.append(new_doc)

print(colloc_docs[3])

## Going beneath the surface (form)

Earlier, we alluded to the process of deciding which tokens to treat as having the same type (for instance, we might decide "We" and "we" are the same type, even though they have different capitalization - a case of *case folding*). There are lots of things that might affect what we decide counts as the same type or not:

* We may want to distinguish the part of speech of a word, so "bank" (the noun) is different from "bank" (the verb)
* We may want to combine different morphologies with the same root or *lemma*, such as treating "lift", "lifts", and "lifted" as the same
* We may want to conflate all punctuations symbols that end sentences as one token (e.g., *PUNCT*)
* We may want to treat all numeric tokens of the same length (e.g. 1975 -> ####, 2 -> #)

For deciding when to do these things, it's always worth thinking about what your research question and how this will affect it. For instance, when deciding whether to conflate words with the same root, you can use a heuristic called a *stemmer* or a lookup table called a *lemmatizer* to replace words with their root forms. If you're interested in when people mention climate change, that's probably reasonable, as you might want to make sure "environment, "environmental", etc. are all treated similarly. However, if you're doing something on comparative religions, treating "god" and "gods" as the same is probably not a good idea! Usually, when in doubt, it's good to be conservative - even if features look like duplicates and make it harder to read through, if you can run your analysis with more features, try doing so first and see if the increase in different word forms is actually interfering with what the model gets out at the document level. (For English topic models, both stopwords and stemmers don't change much!)

A note: when you do processing to change these things, it can affect the steps taken by other processing units. For instance, lemmatizers use surrounding context to figure out the form of a word, so if you change capitalization or remove stopwords, that can mess your lemmatizer up. Luckily, there's a tool that can manage a pipeline of different processing steps for you called `spaCy`, which includes lots of different token-level processing tools and models for different languages. Let's see an example of how `spaCy` represents tokens with a model trained on web and social media data:

In [ ]:
# do *all* the processing!
nlp = spacy.load("en_core_web_sm")
new_data = list(nlp.pipe(clean_newsgroup_data.data))

The above cell will take a bit - that's because spaCy is running a number of automatic analyzers in succession on the text, which tends to take some effort. Once it runs, we can print out a processed `spaCy` `Doc` object for each document:

In [ ]:
example_doc = new_data[27]
print(example_doc, '\n')
type(example_doc)

Well...that looks like a document. What's new? Lots -- but for our current task, the thing we might be excited about is that this document supports iterating through the tokens in the document and finding out lots of information. Let's print out some information about 100 tokens below:

In [ ]:
example_tokens = [tok for tok in example_doc]
for tok in example_tokens[:100]:
    print(tok.idx, tok,
          '\tlemma:', tok.lemma_,
          '\talpha:', tok.is_alpha,
          '\tpunct:', tok.is_punct,
          '\tstopword:', tok.is_stop,
          '\tPOS tag:', tok.pos_)

This is a *lot* to unpack, and isn't even everything, but we can translate these lines to tell us about a lot of properties of a token `tok`:
* `tok.idx` tells us at what index, or position, the first character of that token has within the string (e.g. the "HOW" at the start of "HOW TO BECOME AN ASTRONAUT" is the 73rd character of the document)
* `tok.lemma` tells us the root or lemmatized form of a word (e.g. at index 368, the lemma of "paying" is "pay"). These are inferred using a combination of contextual information and a lookup table. A *stemmer* is a rule-based program that tries to guess lemmas quickly using syntax rules, and will run both faster and with more errors than a lemmatizer - but neither system is perfect.
* `tok.is_alpha` tells us whether the token is all letters or not (punctuation, dates, and times are not alphabetical)
* `tok.is_punct` tells us if a token is punctuation.
* `tok.is_stop` tells us if a token is part of the stoplist for this spaCy language model (here, one built for web and social media, which we can see includes all of "How", "do", "I", "become", and "an" but not "astronaut"
* `tok.pos_` tells us the part of speech of a tag based on the Universal POS tag set, a standard set used in many natural language processing systems and evaluations. These are automatically inferred, sometimes incorrectly - see at index 90, where ASTRONAUT is inferred to be an adjective modifying "First".

..a lot more of these `is_` checks exist for things ranging from whether a token is at the start of end of a sentence to if it's referring to currency or is a piece of left-side punctuation. I recommend checking out the [Token documentation](https://spacy.io/api/token) for the full list. The key of this, though, is that this processing scheme can allow you to rewrite your text into tokens that encode all the information you care about. For instance, if I want to count word types defined based on the intersection of a word's lemma and its part of speech, I could do something like:

In [ ]:
# count the frequency of different types of token based on their lemma and POS tag (e.g. "you_PRON" for the pronoun "you")
pos_token_counts = Counter([tok.lemma_+'_'+tok.pos_ for tok in example_doc if tok.is_alpha])
print(pos_token_counts.most_common(20))

At this point, we've seen how we can break text into the tokens we care about, and recombine and modify those tokens. We also looked at using Counters and plots to quickly grasp the distribution of common quantities in our data and to spot outliers. I haven't answered a lot of questions about what the right process to tokenize and process this text is, because there isn't one set process: it's usually good to keep inspecting as you try things, or to compare several options at once on a few sample documents to get a good sense of what works. But again, when in doubt, be conservative - don't throw information away that you might use.

___
[Proceed to next lesson: Text Data Curation 3/3 ->](./textcuration-3.ipynb)